In [29]:
import pandas as pd
import numpy as np
import datetime as dt

In [30]:
#load the data
dataset = pd.read_csv('../MySKill_SQL_E-Commerce_Analysis-main\order_detail.csv')
dataset.head()

,id,customer_id,order_date,sku_id,price,qty_ordered,before_discount,discount_amount,after_discount,is_gross,is_valid,is_net,payment_id
0,ODR9939707760w,C713589L,2021-11-19,P858068,26100,200,5220000.0,2610000.00,2610000.00,1,1,0,5
1,ODR7448356649d,C551551L,2021-11-19,P886455,1971942,5,9859710.0,2464927.50,7394782.50,1,0,0,5
2,ODR4011281866z,C685596L,2021-11-25,P678648,7482000,1,7482000.0,2065344.62,5416655.38,1,0,0,4
3,ODR3378927994s,C830683L,2021-11-22,P540013,3593680,1,3593680.0,1455440.40,2138239.60,1,1,1,5
4,ODR4904430099k,C191766L,2021-11-21,P491032,4413220,1,4413220.0,1059172.80,3354047.20,1,1,1,4


In [31]:
#convert date with string datatype to datetime datatype
dataset['order_date'] = pd.to_datetime(dataset['order_date'])
dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5884 entries, 0 to 5883
Data columns (total 13 columns):
 #   Column           Non-Null Count  Dtype         
---  ------           --------------  -----         
 0   id               5884 non-null   object        
 1   customer_id      5884 non-null   object        
 2   order_date       5884 non-null   datetime64[ns]
 3   sku_id           5884 non-null   object        
 4   price            5884 non-null   int64         
 5   qty_ordered      5884 non-null   int64         
 6   before_discount  5884 non-null   float64       
 7   discount_amount  5884 non-null   float64       
 8   after_discount   5884 non-null   float64       
 9   is_gross         5884 non-null   int64         
 10  is_valid         5884 non-null   int64         
 11  is_net           5884 non-null   int64         
 12  payment_id       5884 non-null   int64         
dtypes: datetime64[ns](1), float64(3), int64(6), object(3)
memory usage: 597.7+ KB


# TASK 1
Selama transaksi yang terjadi selama 2021, pada bulan apa total nilai transaksi (after_discount) paling besar?

In [32]:
order_1 = np.where((dataset['order_date'].dt.year == 2021) & (dataset['is_valid'] == 1))
order_detail_in_2021 = dataset.loc[order_1]

In [33]:
order_detail_in_2021.groupby(order_detail_in_2021.order_date.dt.month)['after_discount'].sum().sort_values(ascending = False).to_frame()

,after_discount
order_date,
8,2.278627e+08
12,2.173100e+08
10,2.076033e+08
11,1.803960e+08
7,1.480077e+08
9,1.459433e+08
6,4.315455e+07
1,3.682213e+07
2,3.561180e+07


# TASK 2
Selama transaksi yang terjadi selama 2021, pada bulan apa total jumlah pelanggan (unique), total order (unique) dan total jumlah kuantitas produk paling banyak?

In [34]:
order_detail_in_2021.groupby(order_detail_in_2021.order_date.dt.month).agg({
    'customer_id' : ['nunique'],
    'id' : ['nunique'],
    'qty_ordered': ['size']
}).sort_values(by=('customer_id', 'nunique'), ascending=False)

,customer_id,id,qty_ordered
,nunique,nunique,size
order_date,,,
8,236,261,267
12,235,255,258
9,217,238,243
7,214,247,248
10,211,238,242
11,193,209,212
2,56,57,57
6,56,56,58


# TASK 3
Selama transaksi yang terjadi selama 2022, kategori apa yang menghasilkan nilai transaksi paling besar?

In [35]:
order_2022 = np.where((dataset['order_date'].dt.year == 2022) & (dataset['is_valid'] == 1))
order_detail_in_2022 = dataset.loc[order_2022]

In [36]:
sku_detail = pd.read_csv('../MySKill_SQL_E-Commerce_Analysis-main\sku_detail.csv')

In [37]:
order_join_sku = pd.merge(order_detail_in_2022,sku_detail, how = "left", left_on = 'sku_id', right_on = 'id' )

In [38]:
order_join_sku.groupby('category')['after_discount'].sum().sort_values(ascending=False)

category
Mobiles & Tablets     9.184516e+08
Entertainment         3.653441e+08
Appliances            3.163581e+08
Computing             2.140285e+08
Men Fashion           1.355883e+08
Women Fashion         9.301497e+07
Home & Living         7.948372e+07
Health & Sports       5.423558e+07
Beauty & Grooming     4.621102e+07
Superstore            3.264327e+07
Kids & Baby           2.593128e+07
Others                2.174465e+07
Soghaat               1.765833e+07
School & Education    1.736247e+07
Books                 6.792519e+06
Name: after_discount, dtype: float64

# TASK 4
Bandingkan nilai transaksi dari masing-masing kategori pada tahun 2021 dengan 2022. Sebutkan kategori apa saja yang mengalami peningkatan dan kategori apa yang mengalami penurunan nilai transaksi dari tahun 2021 ke 2022.

In [39]:
order_with_sku = pd.merge(dataset,sku_detail, how='left', left_on=['sku_id'], right_on=['id'])

In [40]:
order_with_sku['year'] = order_with_sku['order_date'].dt.year
grouped_order = order_with_sku.groupby(['year', 'category'])['after_discount'].sum().reset_index()

In [41]:
pivoted_order = grouped_order.pivot(index='category', columns='year', values='after_discount')
pivoted_order.rename(columns=lambda x: f"year_{x}", inplace=True)

In [42]:
pivoted_order['diff_by_year'] = (pivoted_order['year_2022'] - pivoted_order['year_2021']).round(2)

In [43]:
def diff_status(x):
    if x > 0:
        return 'Naik'
    else:
        return 'Turun'

pivoted_order['diff_status'] = pivoted_order['diff_by_year'].apply(lambda x:diff_status(x))
pivoted_order.sort_values(by='diff_by_year', ascending=False)

year,year_2021,year_2022,diff_by_year,diff_status
category,,,,
Mobiles & Tablets,1.204388e+09,2.248577e+09,1.044190e+09,Naik
Entertainment,5.133826e+08,8.115787e+08,2.981961e+08,Naik
Appliances,4.803979e+08,7.047408e+08,2.243429e+08,Naik
Men Fashion,1.045421e+08,1.765249e+08,7.198275e+07,Naik
Health & Sports,5.781093e+07,9.185908e+07,3.404815e+07,Naik
Beauty & Grooming,6.942936e+07,1.004874e+08,3.105808e+07,Naik
Home & Living,9.336559e+07,1.186937e+08,2.532816e+07,Naik
Women Fashion,1.226124e+08,1.448668e+08,2.225444e+07,Naik
Soghaat,2.331033e+07,2.963243e+07,6.322099e+06,Naik


# TASK 5
Tampilkan Top 10 sku_name (beserta kategorinya) berdasarkan nilai transaksi yang terjadi selama tahun 2022. Tampilkan juga total jumlah pelanggan (unique), total order (unique) dan total jumlah kuantitas.

In [44]:
order_2022_sku = np.where((order_with_sku['order_date'].dt.year == 2022) & (order_with_sku['is_valid'] == 1))
order_detail_2022 = order_with_sku.loc[order_2022_sku]

In [45]:
order_detail_2022.groupby(['sku_name','category']).agg({
    'after_discount' : ['sum'],
    'id_x' : ['nunique'],
    'customer_id' : ['nunique'],
    'qty_ordered' : ['sum']
}).sort_values(by=('after_discount','sum'), ascending=False).head(10)

,,after_discount,id_x,customer_id,qty_ordered
,,sum,nunique,nunique,sum
sku_name,category,,,,
IDROID_BALRX7-Gold,Mobiles & Tablets,518752000.0,1,1,1000
Samsung_40K5000,Entertainment,60289782.0,1,1,21
iphone_7-128GB-wof-Matt Black,Mobiles & Tablets,34127490.0,6,6,6
HP_MFP M435nw (A3 Blk 3 in 1),Computing,31373244.0,6,4,6
samsung_K6000 49,Entertainment,31339952.0,6,6,6
iphone_7-128GB-wof-Jet Black,Mobiles & Tablets,28535710.0,5,5,5
MACBOOK PRO 13-INCH 2.7GHZ,Computing,26100000.0,2,2,3
Samsung-Galaxy-S8-G955-Plus-Black,Mobiles & Tablets,25853210.0,4,4,5


# TASK 6
Tampilkan top 5 metode pembayaran yang paling populer digunakan selama 2022 (berdasarkan total unique order).

In [46]:
payment_method = pd.read_csv('../MySKill_SQL_E-Commerce_Analysis-main\payment_detail.csv')
payment_method.head()

,id,payment_method
0,1,cod
1,2,jazzvoucher
2,3,customercredit
3,4,Payaxis
4,5,jazzwallet


In [47]:
order_with_payment = pd.merge(dataset, payment_method, how='left', left_on='payment_id', right_on='id')
order_2022 = np.where((order_with_payment['order_date'].dt.year == 2022) & (order_with_payment['is_valid'] == 1))
order_payment_2022 = order_with_payment.loc[order_2022]
order_payment_2022

,id_x,customer_id,order_date,sku_id,price,qty_ordered,before_discount,discount_amount,after_discount,is_gross,is_valid,is_net,payment_id,id_y,payment_method
6,ODR7610732813d,C313534L,2022-12-01,P849301,2697000,1,2697000.0,809100.00,1887900.00,1,1,1,1,1,cod
7,ODR4415476736l,C215074L,2022-12-01,P918122,2533672,1,2533672.0,760101.60,1773570.40,1,1,0,3,3,customercredit
16,ODR8451284001a,C805381L,2022-05-22,P726605,3278450,1,3278450.0,406000.00,2872450.00,1,1,1,4,4,Payaxis
20,ODR2268957100j,C813380L,2022-04-16,P619324,5626000,1,5626000.0,348000.00,5278000.00,1,1,1,7,7,Easypay
26,ODR2463904091r,C869172L,2022-04-15,P873818,841000,2,1682000.0,254887.38,1427112.62,1,1,0,1,1,cod
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5877,ODR6864259419h,C714630L,2022-07-25,P194155,0,3,0.0,0.00,0.00,1,1,0,1,1,cod
5878,ODR8376183747r,C300950L,2022-07-28,P194155,0,1,0.0,0.00,0.00,1,1,0,1,1,cod
5879,ODR4312238651y,C295679L,2022-07-29,P194155,0,2,0.0,0.00,0.00,1,1,0,1,1,cod
5880,ODR3930118778w,C368658L,2022-09-06,P194155,0,1,0.0,0.00,0.00,1,1,1,1,1,cod


In [48]:
order_payment_2022.groupby('payment_method').agg({
    'id_x' : ['nunique']
}).sort_values(by=('id_x','nunique'), ascending=False).head()

,id_x
,nunique
payment_method,
cod,1809
Payaxis,181
customercredit,75
Easypay,69
jazzwallet,26


# TASK 7
Urutkan dari ke-5 produk ini berdasarkan nilai transaksinya yaitu Samsung, Apple, Sony, Huawei, Lenovo

In [49]:
def check_brand(x):
    if 'samsung' in x.lower():
        return 'samsung'
    elif 'sony' in x.lower():
        return 'sony'
    elif 'apple' in x.lower():
        return 'apple'
    elif 'huawei' in x.lower():
        return 'huawei'
    elif 'lenovo' in x.lower():
        return 'lenovo'
    else:
        return 'others'

order_with_sku['brand'] = order_with_sku['sku_name'].apply(check_brand)
only_brand = np.where((order_with_sku['brand'] != 'others') & (order_with_sku['is_valid'] == 1))
grouped_brand = order_with_sku.loc[only_brand]

In [50]:
grouped_brand.groupby('brand').agg({
    'after_discount': ['sum']
}).sort_values(by=('after_discount','sum'), ascending=False)

,after_discount
,sum
brand,
samsung,588764148.0
apple,213310080.0
sony,63960718.0
huawei,63160260.0
lenovo,62379800.4


# TASK 8
Seperti Task 3, buatlah perbandingan dari nilai profit tahun 2021 dan 2022 pada tiap kategori. Kemudian buatlah selisih % perbedaan profit antara 2021 dengan 2022 (profit = after_discount — (cogs*qty_ordered))

In [81]:
order_with_sku['profit'] = order_with_sku['after_discount'] - (order_with_sku['cogs'] * order_with_sku['qty_ordered'])
grouped_profit = order_with_sku.groupby(['year','category'])['profit'].sum().reset_index()


In [ ]:
pivoted_profit = grouped_profit.pivot(index='category', columns='year', values='profit')
pivoted_profit.rename(columns=lambda x: f"year_{x}", inplace=True)
pivoted_profit

In [88]:
pivoted_profit['profit_diff'] = (100 * (pivoted_profit['year_2022'] - pivoted_profit['year_2021']) / pivoted_profit['year_2021'] ).round()
pivoted_profit.sort_values(by='profit_diff', ascending=False)

year,year_2021,year_2022,profit_diff
category,,,
Health & Sports,1.381434e+07,2.265493e+07,64.0
Beauty & Grooming,1.787369e+07,2.717080e+07,52.0
School & Education,3.521400e+06,5.291377e+06,50.0
Appliances,1.201006e+08,1.784516e+08,49.0
Entertainment,1.323704e+08,1.915733e+08,45.0
Soghaat,5.275987e+06,7.601654e+06,44.0
Men Fashion,2.812455e+07,3.915922e+07,39.0
Women Fashion,2.897985e+07,3.556501e+07,23.0
Mobiles & Tablets,3.066036e+08,3.404876e+08,11.0


# TASK 9
Tampilkan top 5 SKU dengan kontribusi profit paling tinggi di tahun 2022 berdasarkan kategori paling besar pertumbuhan profit dari 2021 ke 2022 (berdasarkan hasil Task 8).

In [133]:
health_category = np.where((order_with_sku['category'] == 'Health & Sports') & (order_with_sku['order_date'].dt.year == 2022) & (order_with_sku['is_valid'] == 1))
health_category_profit = order_with_sku.loc[health_category]

False

In [131]:
health_category_profit[['sku_name', 'profit']].sort_values(by='profit', ascending=False).head(5)

,sku_name,profit
1157,Gmart_Forerunner735XT 3,1160000.0
741,Ifitness_T300,998760.0
1174,MORG_GCS 41,682080.0
149,Ifitness_JS9.4A,453850.0
1864,MORG_GCS 26,386860.0


# TASK 10
Tampilkan jumlah unique order yang menggunakan top 5 metode pembayaran (Task 6) berdasarkan kategori produk selama tahun 2022.

In [ ]:
df1 = pd.merge(dataset, sku_detail, how='left', left_on='sku_id', right_on='id')
order_sku_payment = pd.merge(df1, payment_method, how='left', left_on='payment_id', right_on='id')

In [ ]:
payment = np.where((order_sku_payment['order_date'].dt.year == 2022) & (order_sku_payment['is_valid'] == 1))
payment_2022 = order_sku_payment.loc[payment]

In [ ]:
grouped_payment = payment_2022.groupby(['category', 'payment_method'])['id_x'].nunique().reset_index()

In [204]:
pivoted_payment = grouped_payment.pivot(index='category', columns='payment_method', values='id_x')
pivoted_payment[['cod','Payaxis','customercredit','Easypay','jazzwallet']].sort_values(by='cod', ascending=False)

payment_method,cod,Payaxis,customercredit,Easypay,jazzwallet
category,,,,,
School & Education,157.0,7.0,1.0,1.0,2.0
Home & Living,152.0,10.0,12.0,4.0,1.0
Superstore,145.0,15.0,7.0,4.0,9.0
Others,144.0,13.0,5.0,4.0,4.0
Health & Sports,138.0,15.0,4.0,5.0,1.0
Kids & Baby,137.0,15.0,7.0,10.0,3.0
Soghaat,128.0,11.0,6.0,6.0,2.0
Men Fashion,127.0,11.0,8.0,7.0,NaN
Books,126.0,5.0,2.0,NaN,1.0
